In [1]:
import pandas as pd
import numpy as np
import urllib
import os
import zipfile
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from copy import deepcopy
import tqdm

In [2]:
def make_dir_if_not_present(folder):
    if os.path.exists(folder):
        pass
    else:
        os.mkdir(folder)

In [33]:
make_dir_if_not_present("data")
make_dir_if_not_present("results")
make_dir_if_not_present("figures")
make_dir_if_not_present("scripts")
make_dir_if_not_present("trained_models")
make_dir_if_not_present("studies")

In [4]:
def download_if_not_present(url, destination):
    if os.path.exists(destination):
        pass
    else:
        urllib.request.urlretrieve(url, destination)

In [30]:
# Download GDSC1 and GDSC2
download_if_not_present("https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.5/GDSC1_fitted_dose_response_27Oct23.xlsx",
                       "data/GDSC1.xlsx")
download_if_not_present("https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.5/GDSC2_fitted_dose_response_27Oct23.xlsx",
                       "data/GDSC2.xlsx")

# Download compound and cell-line identifiers
download_if_not_present("https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.5/Cell_Lines_Details.xlsx",
                       "data/Cell_Lines_Details.xlsx")
download_if_not_present("https://cog.sanger.ac.uk/cancerrxgene/GDSC_release8.5/screened_compounds_rel_8.5.csv",
                       "data/screened_compounds_rel_8.5.csv")
download_if_not_present("https://cog.sanger.ac.uk/cmp/download/model_list_20240110.csv",
                       "data/cell_line_annotations.csv")

# Download omics features
download_if_not_present("https://cog.sanger.ac.uk/cmp/download/mutations_all_20230202.zip",
                       "data/raw_mutations.zip")

download_if_not_present("https://cog.sanger.ac.uk/cmp/download/rnaseq_all_20220624.zip",
                       "data/rnaseq.zip")

download_if_not_present("https://cog.sanger.ac.uk/cmp/download/Proteomics_20221214.zip",
                       "data/proteomics.zip")

download_if_not_present("https://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources//Data/BEMs/CellLines/CellLines_METH_BEMs.zip",
                       "data/methylation.zip")

download_if_not_present("https://cog.sanger.ac.uk/cmp/download/driver_genes_20221018.csv",
                       "data/driver_genes.csv")
download_if_not_present("https://cog.sanger.ac.uk/cmp/download/driver_mutations_20221208.csv",
                       "data/driver_mutations.csv")

In [6]:
# Preprocessing mutations. 
# All mutations are treated simply as loss of function without accounting for differences, which might not be accurate
mutations = pd.read_csv("data/raw_mutations.zip")
mutations_filtered = mutations.query("coding == True & effect != 'silent'").loc[:, ["gene_symbol", "model_id"]].assign(mutated=1)
mutations_matrix = mutations_filtered.groupby(["model_id", "gene_symbol"]).max().unstack().fillna(0)
mutations_matrix.to_csv("data/binary_mutations.csv")

In [7]:
# Preprocessing proteometics
zipf = zipfile.ZipFile("data/proteomics.zip")
zipf.extractall("data/")
protein_intensities = pd.read_csv("data/proteomics_all_20221214.csv").groupby([ "model_id", "uniprot_id"])["protein_intensity"].median().unstack()

In [8]:
protein_filtered = protein_intensities.loc[:, protein_intensities.isna().sum(0)< 10] # filter proteins that have a low number of missing values

In [9]:
protein_filtered

uniprot_id,A0FGR8,A1L0T0,A5YKK6,A6NHQ2,A6NHR9,E9PAV3,E9PRG8,O00116,O00154,O00165,...,Q9Y676,Q9Y678,Q9Y679,Q9Y697,Q9Y6C9,Q9Y6D9,Q9Y6E2,Q9Y6I9,Q9Y6M9,Q9Y6W5
model_id,,,,,,,,,,,,,,,,,,,,,
SIDM00018,4.68643,2.67339,4.73981,NaN,4.42869,5.88802,3.86184,4.60401,5.849610,3.42882,...,2.86848,5.29795,2.72116,3.21178,4.88265,4.05882,5.59875,2.48290,3.43450,5.07219
SIDM00023,3.97764,4.25379,4.13866,5.30761,3.62630,5.01270,4.42488,4.07084,4.899690,3.79563,...,4.20974,4.29555,3.25246,4.41326,5.09241,4.35411,5.18875,3.18724,4.49440,4.86868
SIDM00040,4.53991,4.42006,4.16105,4.87183,3.30398,6.62334,4.42636,3.76346,5.943520,3.32730,...,3.08379,5.89432,3.02638,4.35204,5.65149,3.16955,5.23238,2.28483,2.77586,3.77569
SIDM00041,4.40464,3.45449,4.53302,4.64816,3.75523,5.75613,4.59370,3.81364,6.073450,3.00717,...,3.75049,4.90517,2.63217,3.91528,5.24708,4.41301,4.64874,2.39623,4.45386,4.57432
SIDM00042,3.53864,3.55892,NaN,5.63237,3.98601,NaN,2.54964,4.94454,0.716592,3.85426,...,4.26742,1.87530,3.28659,4.58699,5.59893,2.07564,NaN,2.91875,3.41980,2.40811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SIDM01248,2.92145,4.84742,3.89554,5.10445,3.26574,6.07755,5.21614,4.44987,5.669400,4.15336,...,3.90669,5.57841,3.57178,4.09472,4.00083,4.30312,5.58492,3.24431,3.83791,3.77094
SIDM01251,3.75004,1.47213,4.35432,5.05156,3.22245,6.21646,3.53687,3.60728,7.144720,3.07613,...,2.46576,5.16812,2.29456,3.17794,4.41413,3.49614,4.52184,1.95947,2.57177,4.03652
SIDM01259,2.37653,3.16876,4.24307,5.57185,3.27437,6.20949,4.11735,3.45889,5.054950,3.84379,...,2.94827,5.09663,2.42821,3.57565,4.71374,3.80054,5.58092,2.05242,3.11909,4.35954


In [10]:
proteins = protein_filtered.columns

In [11]:
protein_input = deepcopy(protein_filtered)

In [12]:
# Note that for this inputation the pipeline leaks at several points
protein_filtered_copy = deepcopy(protein_filtered)
protein_input = deepcopy(protein_filtered)
medians = protein_filtered.median()
protein_filtered_copy = protein_filtered.fillna(medians)
for protein in tqdm.tqdm(proteins):
    target_inputation = protein_filtered.loc[:, protein]
    if target_inputation.isna().sum() > 0:
        X = protein_filtered_copy.loc[~target_inputation.isna()]
        X_input = protein_filtered_copy.loc[target_inputation.isna()]
        y = X.loc[:, protein]
        X = X.drop(protein, axis=1)
        grid = GridSearchCV(estimator = Ridge(),
                             param_grid = {"alpha": [0.1, 1, 100, 200, 500, 1000, 5000, 10000]},
                             scoring = "r2",
                             n_jobs = -1)
        grid.fit(X, y)
        rdg = Ridge(**grid.best_params_)
        rdg.fit(X, y)
        y_input = rdg.predict(X_input.drop(protein, axis=1))
        protein_input.loc[target_inputation.isna(), protein] = y_input
protein_input.to_csv("data/proteomics.csv")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1642/1642 [12:19<00:00,  2.22it/s]


In [13]:
# Preprocessing methylation
zipf = zipfile.ZipFile("data/methylation.zip")
zipf.extractall("data/")
pd.read_csv("data/METH_CELLLINES_BEMs/PANCAN.txt", sep = "\t").to_csv("data/methylations.csv")

In [14]:
# Preprocessing expression
zipf = zipfile.ZipFile("data/rnaseq.zip")
zipf.extractall("data/")

In [15]:
rnaseq = pd.read_csv("data/rnaseq_read_count_20220624.csv")

/tmp/ipykernel_24369/4037027244.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,2

In [16]:
rnaseq = rnaseq.iloc[4:, 1:].set_index("Unnamed: 1")

In [17]:
rna_filtered = rnaseq.loc[rnaseq.isna().sum(1) == 0] # we remove genes with nans

In [18]:
rna_filtered = rna_filtered.replace(" [0]*", "", regex = True)

In [19]:
rna_filtered = (rna_filtered.T).astype(float)

In [20]:
rna_log = np.log(rna_filtered + 1)

In [21]:
rna_count_norm = (rna_log - rna_log.mean(0))/rna_log.std(0) # by machine learning standards data leakage

In [22]:
rna_count_norm.dropna(axis=1).to_csv("data/rnaseq_normcount.csv") # remove columns with zero variance and save

In [28]:
GDSC1 = pd.read_excel("data/GDSC1.xlsx")
GDSC1.loc[:, ["SANGER_MODEL_ID", "DRUG_ID", "LN_IC50"]].to_csv("data/GDSC1.csv")